In [52]:
import os
import pandas as pd
import numpy as np
from PIL import Image

In [53]:
class_labels = {"Jade":0, "James":1, "Jane":2, "Joel":3, "Jovi":4}

def one_hot_encode(labels):
    one_hot = np.zeros((len(labels), len(class_labels)))
    for i, label in enumerate(labels):  
        one_hot[i, label] = 1 
    return one_hot
    
def load_image(image_path):
    image = Image.open(image_path).convert("L") # open and converting to grayscale
    return np.array(image, dtype=np.float32).flatten()/255 # get image pixels, make an array of the pixels and normalize 

def extract_images_from_folder(folder_name):
    image_data = []
    category_labels = []

    for root, _, files in os.walk(folder_name): # root is the folder train, _ are the subdirectories
        folder_name = os.path.basename(root) # basename extracts category/folder name 
        for name in files:
            if name.endswith(".png"):
                image_path = os.path.join(root, name)
                image_data.append(load_image(image_path))
                category_labels.append(class_labels[folder_name]) # assigns integer according to folder name
    image_data = np.array(image_data)
    category_labels = np.array(category_labels)
    one_hot_labels = one_hot_encode(category_labels)

    indices = np.random.permutation(len(image_data)) # shuffle dataset
    return image_data[indices], one_hot_labels[indices]


In [54]:
def relu(Z):
    return np.maximum(0, Z)

def relu_derivative(Z):
    return Z > 0

def softmax(Z):
    # return np.exp(Z)/np.sum(np.exp(Z)) overflow
    expZ = np.exp(Z - np.max(Z, axis=0, keepdims=True)) # subtracting with largest value makes largest element zero
    return expZ / np.sum(expZ, axis=0, keepdims=True)

def initialize_parameters(input_size, hidden_size, output_size):
    w1 = np.random.randn(hidden_size, input_size) * 0.01 # randn normal distribution (mean=0, std=1), so values centered around 0
    b1 = np.zeros((hidden_size, 1))
    w2 = np.random.randn(output_size, hidden_size) * 0.01
    b2 = np.zeros((output_size, 1))
    return w1, b1, w2, b2

def forward_propagation(w1, b1, w2, b2, X):
    Z1 = np.dot(w1, X) + b1
    A1 = relu(Z1)
    Z2 = np.dot(w2, A1) + b2
    A2 = softmax(Z2)
    return Z1, A1, Z2, A2

def back_propagation(Z1, A1, Z2, A2, w1, w2, X, Y):
    m = Y.shape[1]
    dZ2 = A2 - Y
    dw2 = (1/m) * np.dot(dZ2, A1.T)
    db2 = (1/m) * np.sum(dZ2, axis=1, keepdims=True)
    dZ1 = np.dot(w2.T, dZ2) * relu_derivative(Z1)
    dw1 = (1/m) * np.dot(dZ1, X.T)
    db1 = (1/m) * np.sum(dZ1, axis=1, keepdims=True)
    return dw1, db1, dw2, db2

def update_parameters(w1, b1, w2, b2, dw1, db1, dw2, db2, learning_rate):
    w1 -= learning_rate * dw1
    b1 -= learning_rate * db1
    w2 -= learning_rate * dw2
    b2 -= learning_rate * db2
    return w1, b1, w2, b2

def get_predictions(A2):
    return np.argmax(A2, axis=0) # returns most probable class

def gradient_descent(X, Y, learning_rate, iterations):
    input_size = X.shape[0]
    hidden_size = 5  # number of labels
    output_size = Y.shape[0]
    
    w1, b1, w2, b2 = initialize_parameters(input_size, hidden_size, output_size)
    
    for i in range(iterations):
        Z1, A1, Z2, A2 = forward_propagation(w1, b1, w2, b2, X)
        dw1, db1, dw2, db2 = back_propagation(Z1, A1, Z2, A2, w1, w2, X, Y)
        w1, b1, w2, b2 = update_parameters(w1, b1, w2, b2, dw1, db1, dw2, db2, learning_rate)

    return w1, b1, w2, b2

train_data, train_labels = extract_images_from_folder("Train") # load dataset

w1, b1, w2, b2 = gradient_descent(train_data.T, train_labels.T, 0.10, 500) # train the neural network

In [55]:
def compute_accuracy(predictions, true_labels):
    true_classes = np.argmax(true_labels, axis=1)  # convert one hot to integer labels
    accuracy = np.mean(predictions == true_classes) * 100 
    return accuracy

test_data, test_labels = extract_images_from_folder("Test")

Z1, A1, Z2, A2 = forward_propagation(w1, b1, w2, b2, test_data.T)

predictions = get_predictions(A2)  

accuracy = compute_accuracy(predictions, test_labels)
print(f"Test Accuracy: {accuracy:.2f}%")

Test Accuracy: 94.30%
